# Final Project of Quantum Computation

20195048 김현성

## Problem 1

Solve the following linear system with HHL algorithm

$$\begin{bmatrix}1 & -1/3 \\ -1/3 & 1\end{bmatrix} \mathbf{x} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}$$

## Problem 2 

Extend the problem 1 result and solve following equation

$$\frac{1}{4}
\begin{bmatrix}
15 & 9 & 5 & -3 \\
9 & 15 & 3 & -5 \\
5 & 3 & 15 & -9 \\
-3 & -5 & -9 & 15\end{bmatrix} \mathbf{x} = 
\frac{1}{2} 
\begin{bmatrix}
1\\
1\\
1\\
1
\end{bmatrix}$$

## Answer

Since the matrix already a Hermit matrix, the calculation does not require a conversion. Before we implement HHL. Let us see the exact solutions of the each system.

### Exact solutions

In [7]:
from IPython.display import display, Math

In [8]:
import sympy as sp
from sympy.printing.mathml import mathml
sp.init_printing(use_unicode=True) # allow LaTeX printing


In [9]:
scale1 = 3
A1 = sp.Matrix([[3, -1], [-1, 3]])/scale1
b1 = sp.Matrix([[0], [1]])
scale2 = 4
A2 = sp.Matrix(
    [
        [15, 9, 5, -3], 
        [9, 15, 3, -5],
        [5, 3, 15, -9],
        [-3, -5, -9, 15]])/scale2
b2 = sp.Matrix(4*[[2]])/scale2

In [20]:
display(Math(sp.latex(A1)))
display(Math(f"$det(A_1): {A1.det()}$",))
display(Math("$\mathbf{x}: " + sp.latex(A1.solve(b1).T)))
display(Math(sp.latex(A2)))
display(Math(f"$det(A_2): {A2.det()}$",))
display(Math("$\mathbf{x}: " + sp.latex(A2.solve(b2).T)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Implementation of HHL algorithm

Major steps of HHL algorithm

3 registers required: ancilla, main-register, b-register.

1. State preparation: Encoding $|b\rangle$ vector to the register, amplitude encoding(b).
2. Quantum Phase estimation(main, b).
3. Ancilla bit rotation(main, ancilla).
4. Inverse Quantum Phase estimation(main, b).
5. Measurement.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

In [21]:
num_ancilla_register = 1
num_main_register = 4
num_vector_register = 4 

In [22]:
qr_ancilla = QuantumRegister(num_ancilla_register, name="ancilla")
qr_main = QuantumRegister(num_main_register, name="main")
qr_vector = QuantumRegister(num_vector_register, name="vector")
cr = ClassicalRegister(1, name = "classic")

In [23]:
qc = QuantumCircuit(qr_ancilla, qr_main, qr_vector, cr, name="HHL")

State preparation for $|b\rangle$ vector.

In [24]:
from qiskit.circuit.library import StatePreparation

In [25]:
b1

In [ ]:
def initiate_matrix(A):
    #return A_gates
    pass
def HHL(A, wires):
    #return qc
    pass

### Results